### sklearn.linear_model.LinearRegression
#### 연속형 원인변수가 연속형 결과변수에 영향을 미치는지를 분석하여 레이블 변수 예측

#### 평가지표
- SSE : 오차제곱합(예측값과 실제값의 차이의 제곱합)
- AE : 평균오차(예측값과 평균의 차이로, 예측값이 평균적으로 미달/초과하는지 확인)
- MSE : 평균제곱오차(예측오차 제곱합의 평균)
- MAE : 평균절대오차(예측오차 절대값의 평균)
- RMSE : 평균제곱근오차(종속변수와 단위가 같아 설명이 쉽고 표준편차와 같이 예측에 대한 오차 정보 제공)
- MAPE : 평균절대백분율오차(실제값에 대한 오차의 백분율로, 시계열분석에서 예측 성능지표로 자주 활용)

##### LinearRegression(*, fit_intercept=True, normalize=False, copy_X=True, n_jobs=None, positive=False)

# 분석 코드

In [1]:
# 라이브러리 및 데이터 로드
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error

df = pd.read_csv('../input/big-data-certification-study/house_price.csv', encoding='utf-8')
df.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [2]:
# 데이터셋 분리
X = df.drop(columns=['house_value'])
y = df[['house_value']]

df.shape, X.shape, y.shape

((17689, 6), (17689, 5), (17689, 1))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# stratify : y 범주의 비율에 따라 데이터 분리

## statsmodels 적용
- 각종 통계량 및 지표 확인 모델

In [4]:
import statsmodels.api as sm
x_train_new=sm.add_constant(X_train)
x_test_new=sm.add_constant(X_test)
x_train_new.head()

,const,housing_age,income,bedrooms,households,rooms
17235,1.0,12,2.0577,0.185449,3.945455,6.372727
14220,1.0,37,4.0000,0.171566,2.741497,6.363946
3280,1.0,24,5.8904,0.154485,2.969325,6.651840
15279,1.0,17,0.9393,0.241460,3.257256,4.518470
14727,1.0,16,2.7143,0.194977,2.679287,6.385301


In [5]:
multi_model=sm.OLS(y_train,x_train_new).fit()
print(multi_model.summary())

                            OLS Regression Results                            
Dep. Variable:            house_value   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     3525.
Date:                Tue, 28 Jun 2022   Prob (F-statistic):               0.00
Time:                        16:02:57   Log-Likelihood:            -1.6533e+05
No. Observations:               13266   AIC:                         3.307e+05
Df Residuals:                   13260   BIC:                         3.307e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.092e+05   9108.171    -11.985      

- R-squared(설명력) : 54.6% 수준으로 회귀직선(예측값)과 실제값 일치
- coef : X 변수가 1 증가할 때 Y 변수가 변화하는 정도로, 기울기
- p<|t| : 통계적으로 유의한가를 검증한 결과로, p-value가 0.05보다 작을 경우 유의한 영향을 미치는 변수로 해석 가능

## sklearn 적용

In [6]:
# 정규화 - Min-Max Scaling
scaler=MinMaxScaler()
scaler.fit(X_train) # feature data의 train data만 fit
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [7]:
# 모델 적용/예측 및 정확도
model=LinearRegression()
model.fit(scaled_X_train, y_train)
pred_x = model.predict(scaled_X_train)
model.score(scaled_X_train, y_train)

0.5706921210926263

In [8]:
pred_y = model.predict(scaled_X_test)
model.score(scaled_X_test, y_test)

0.5826083517811865

In [9]:
# RMSE(Root Mean Squared Error)
rmse_train=np.sqrt(mean_squared_error(y_train,pred_x)) 
rmse_test=np.sqrt(mean_squared_error(y_test,pred_y)) 
print('Train RMSE :', round(rmse_train), '\nTest  RMSE :', round(rmse_test))

Train RMSE : 62537 
Test  RMSE : 61764


In [10]:
# MAE(Mean Absolute Error)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, pred_y)

46218.35635910518

In [11]:
# MSE(Mean Squared Error)
mean_squared_error(y_test, pred_y)

3814773021.730826

In [12]:
# MAPE(Mean Absolute Percentage Error)
abs((y_test-pred_y)/y_test).mean()*100

house_value    30.33032
dtype: float64

In [13]:
# MPE(Mean Percentage Error)
((y_test-pred_y)/y_test).mean()*100

house_value   -12.178049
dtype: float64